In [ ]:
%config Completer.use_jedi = False
# !pip install timm
!pip install torchsummary

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from matplotlib.image import imread
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
from torchsummary import summary
# import timm
from tqdm import tqdm
import os, sys, gc
import time

############
data_path = '../input/cassava-leaf-disease-classification/train_images/'

In [ ]:
# Check GPU is available or not
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
total_csv = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
total_csv.head()

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

In [ ]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

In [ ]:
def read_dataset(path, df):
    """path:`../---/--/`"""
    img_array = []
    for img_name in tqdm(df['image_id']):
        full_path = path + img_name
        img = get_img(full_path).copy()
        img = torch.tensor(img).permute(2,0,1)  # convert to torch format color
        img_array.append(img)
    img_array = torch.stack(img_array)
    return img_array

imm = cv2.imread('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
print(imm.shape)
# data_raw = read_dataset(data_path, train_csv)
# print(data_raw.shape)

In [ ]:
class HDD_Dataset(Dataset):
    def __init__(self, df, data_root, device, transforms=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.device = device
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        target = torch.tensor(self.df.iloc[index]['label'], device=device)
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        img = torch.tensor(get_img(path).copy()).float().to(device)/255  # normalize to 1
        img = img.permute(2,0,1)
        if self.transforms:
            img = self.transforms(img)
        return {'image':img, 'label':target}


class RAM_Dataset(Dataset):
    def __init__(self, df, imgs, device, transforms=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.imgs = imgs
        self.device = device
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        target = self.df.iloc[index]['label']
        img  = self.imgs[index].float().to(device)/255  # normalize to 1
        if self.transforms:
            img = self.transforms(img)
        return {'image':img, 'label':target}

## split data to train, val section

In [ ]:
def frac_train_val(N:int, val_frac:float):
    """N: dataset length e.g. 2100
        val_frac: validation fraction of total e.g. 0.2 
        return indices of train, validation"""
    perm = np.random.permutation(N)
    thrshld = int(N*val_frac)
    return perm[thrshld:].tolist(), perm[:thrshld].tolist()


transform_train = transforms.Compose([
    transforms.RandomResizedCrop(500),
    transforms.ColorJitter(hue=.07, saturation=.08),
    transforms.RandomAffine(25, (0.1, 0.1), (0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomCrop(400)])

transform_val = transforms.Compose([transforms.RandomResizedCrop(400)])  # ??
########### split indices
inds_train, inds_val = frac_train_val(total_csv.shape[0], 0.2)
# val_imgs = read_dataset(data_path, train_csv.loc[inds_val])  # store in `int` to reduce memory usage
train_csv = total_csv.loc[inds_train]
val_csv = total_csv.loc[inds_val]
#create datasets
train_dataset = HDD_Dataset(train_csv, data_path, device, transform_train)

val_dataset = HDD_Dataset(val_csv, data_path, device, transform_val)
gc.collect()

In [ ]:
class print_layer(nn.Module):
    def __init__(self):
        super(print_layer, self).__init__()
    def forward(self, x):
        print(x.shape)
        return x


class miniResNet(nn.Module):
    def __init__(self, C_in, C_mid):
        super(miniResNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(C_in, C_mid, kernel_size=1), nn.ReLU(),
            nn.Conv2d(C_mid, C_mid, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(C_mid, C_in, kernel_size=1), nn.ReLU()
        )
    
    def forward(self, x):
        return self.model(x) + x


class myClassifier(nn.Module):
    def __init__(self,):
        super(myClassifier, self).__init__()
        self.seq = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2), nn.ReLU(),  #200
        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), nn.ReLU(),
        nn.Dropout2d(0.8),
        nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2), nn.ReLU(),
        nn.MaxPool2d(2, 2),  # 100
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128), nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Dropout2d(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.MaxPool2d(2, 2), # 50
        miniResNet(128, 50),miniResNet(128, 50),miniResNet(128, 50),miniResNet(128, 50),
        miniResNet(128, 50),miniResNet(128, 50),# print_layer(),
        nn.AvgPool2d(50),# print_layer()
        )
        self.Lin = nn.Sequential(
        nn.Linear(128, 30), nn.ReLU(),
        nn.Linear(30, 5), nn.ReLU()#, print_layer()
        )
        
    def forward(self, x):
        N = x.shape[0]
        x_conv = self.seq(x).reshape(N, 128)
        x_lin = self.Lin(x_conv)
        return x_lin

In [ ]:
x = torch.randn(2, 3,400,400)
m = myClassifier()
y = m(x)

## prequisites of optimizing

In [ ]:
@torch.no_grad()
def Evaluation(model, test_dataset, path, batch_size, device, criterion):
    model.eval()
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)
    acc = 0
    loss = 0  # torch.tensor(0)
    for data_batch in tqdm(test_dataloader):
        score_pred = model(data_batch['image'])
        y_pred = torch.max(score_pred, dim=1)[1]
        acc = acc + torch.sum(y_pred == data_batch['label'])
        loss += criterion(score_pred, data_batch['label']).data
    acc = acc / (len(test_dataloader)*batch_size)
    loss = loss / len(test_dataloader)
    model.train()
    return acc, loss

In [ ]:
# Create model instance
model = myClassifier()
model = model.to(device, non_blocking=True)  # to GPU or CPU
summary(model, input_size=(3,400,400))
# Define hyperparameters
lr = torch.tensor(0.0001).to(device)
lr_decay = torch.tensor(0.95).to(device)  # per epoch
lr_floor = torch.tensor(0.00001).to(device)
# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Hyper parameters
batch_size = 64
n_epochs = 5
# dataloader
train_dataloader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
tot_batch = len(train_dataloader)
# loss
train_loss, train_acc = [], []
val_loss, val_acc = [], []
# Training RNN
for epoch in range(1, n_epochs + 1):
    epoch_start = time.time()
    for num_batch, data_batch in enumerate(train_dataloader):
        batch_start = time.time()
        
        optimizer.zero_grad()
        output = model(data_batch['image'])
        loss = criterion(output, data_batch['label'])
        loss.backward()
        optimizer.step()
        acc = torch.sum(torch.max(output, dim=1)[1] == data_batch['label']) / batch_size
        
        batch_stop = time.time()
        print(f"*****Epoch {epoch}, minibatch: {num_batch+1}/{tot_batch} elapsed {batch_stop-batch_start:0.1f}s, loss:{loss:0.4f}*****")  # write in replace
    train_loss.append(loss)
    train_acc.append(acc)
    tmp_acc, tmp_loss = Evaluation(model, val_dataset, data_path, batch_size, device, criterion)
    val_loss.append(tmp_loss)
    val_acc.append(tmp_acc)
    epoch_stop = time.time()
    print(f"Epoch {epoch}/{n_epochs} has finished in {epoch_stop-epoch_start:0.1f}s, train loss: {loss:0.4f}, train acc: {acc*100:0.2f}, val loss: {tmp_loss:0.4f},  val acc:{tmp_acc*100:0.2f}")
    lr *= lr_decay
    if lr < lr_floor:
        lr = lr_floor
    gc.collect()

In [ ]:
torch.save(model.state_dict(),'./model_1.h5')

In [ ]:
torch.save(model,'./model_t_1.h5')

In [ ]:
fig, ax = plt.subplots(1,2)
####
ax[0].plot(range(1,len(train_loss)+1), train_loss)
ax[0].plot(range(1,len(val_loss)+1), val_loss)
ax[0].set(xlabel='epoch')
ax[0].set_title('loss')
ax[0].legend(('train','val'))

ax[1].plot(range(1,len(train_loss)+1), train_acc)
ax[1].plot(range(1,len(val_loss)+1), val_acc)
ax[1].set(xlabel='epoch')
ax[1].set_title('accuracy')
ax[1].legend(('train','val'))
plt.show()